In [1]:
import pandas as pd
import numpy as np
import pickle

In [2]:
data = pd.read_csv('pdmssetw.csv')
data

,Unnamed: 0,MeetTijd,HR(bpm),SpO2(%),NIBP(mmHg),IcNaOk
0,0,[''],[''],[''],[''],0.0
1,1,[''],[''],[''],[''],0.0
2,2,[''],[''],[''],[''],0.0
3,3,[''],[''],[''],[''],0.0
4,4,[''],[''],[''],[''],0.0
...,...,...,...,...,...,...
45537,45537,"['19:14:19', '19:14:49', '19:15:19', '19:15:49...","['', '80', '81', '76', '83', '84', '79', '88',...","['100', '100', '100', '100', '100', '100', '10...","['', '123', '', '', '', '', '', '', '', '', ''...",0.0
45538,45538,"['15:17:54', '15:18:24', '15:18:54', '15:19:00...","['67', '', '68', '68', '68', '68', '71', '67',...","['97', '98', '98', '97', '95', '91', '98', '98...","['', '', '', '', '115', '', '', '', '', '113',...",0.0
45539,45539,"['17:39:54', '17:40:24', '17:40:54', '17:41:24...","['100', '103', '97', '101', '100', '106', '107...","['99', '99', '99', '99', '99', '99', '100', '9...","['', '', '', '', '', '', '', '', '', '', '', '...",0.0
45540,45540,"['19:24:26', '19:24:56', '19:25:25', '19:25:56...","['74', '75', '75', '74', '69', '73', '73', '72...","['98', '98', '96', '96', '97', '98', '98', '98...","['178', '', '', '', '', '', '', '185', '', '',...",1.0


In [3]:
data = data.drop(columns=['Unnamed: 0'])

In [4]:
np.nansum(data['IcNaOk'])/len(data)

0.08104606736638707

In [5]:
import ast
data.MeetTijd = data.MeetTijd.apply(ast.literal_eval)
data['HR(bpm)'] = data['HR(bpm)'].apply(ast.literal_eval).apply(lambda x: np.array(x))
data['SpO2(%)'] = data['SpO2(%)'].apply(ast.literal_eval).apply(lambda x: np.array(x))
data['NIBP(mmHg)'] = data['NIBP(mmHg)'].apply(ast.literal_eval).apply(lambda x: np.array(x))

In [6]:
data['Length_MeetTijd'] = data['MeetTijd'].apply(lambda x: len(x))

In [7]:
data = data[data['Length_MeetTijd'] > 3]
data = data.dropna()

In [8]:
data = data.reset_index(drop=True)

In [9]:
data = data[data['Length_MeetTijd'] <2000]

In [10]:
data['HR(bpm)'] = data['HR(bpm)'].apply(lambda x: np.where(x=='', np.nan, x))
data['SpO2(%)'] = data['SpO2(%)'].apply(lambda x: np.where(x=='', np.nan, x))
data['NIBP(mmHg)'] = data['NIBP(mmHg)'].apply(lambda x: np.where(x=='', np.nan, x))

In [11]:
data['Percentage HR nan'] = data['HR(bpm)'].apply(lambda x: np.isnan(x.astype(float)).sum()/len(x))
data['Percentage SpO2 nan'] = data['SpO2(%)'].apply(lambda x: np.isnan(x.astype(float)).sum()/len(x))
data['Percentage NIBP nan'] = data['NIBP(mmHg)'].apply(lambda x: np.isnan(x.astype(float)).sum()/len(x))

In [12]:
data90 = data[(data['Percentage HR nan']<0.10) & (data['Percentage SpO2 nan']<0.10) & (data['Percentage NIBP nan']<0.10)]
data95 = data[(data['Percentage HR nan']<0.05) & (data['Percentage SpO2 nan']<0.05) & (data['Percentage NIBP nan']<0.05)]
data99 = data[(data['Percentage HR nan']<0.01) & (data['Percentage SpO2 nan']<0.01) & (data['Percentage NIBP nan']<0.01)]
data100 = data[(data['Percentage HR nan']==0) & (data['Percentage SpO2 nan']==0) & (data['Percentage NIBP nan']==0)]

In [13]:
pickle.dump(data, open("clean_data.pk", "wb"))
pickle.dump(data100, open("data100.pk", "wb"))

In [148]:
data90['HR(bpm)'] = data90['HR(bpm)'].apply(lambda x: np.array(pd.Series(x,dtype=float).interpolate().fillna(method='bfill').fillna(method='ffill')))
data90['SpO2(%)'] = data90['SpO2(%)'].apply(lambda x: np.array(pd.Series(x,dtype=float).interpolate().fillna(method='bfill').fillna(method='ffill')))
data90['NIBP(mmHg)'] = data90['NIBP(mmHg)'].apply(lambda x: np.array(pd.Series(x,dtype=float).interpolate().fillna(method='bfill').fillna(method='ffill')))

<ipython-input-148-c1fb62223757>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data90['HR(bpm)'] = data90['HR(bpm)'].apply(lambda x: np.array(pd.Series(x,dtype=float).interpolate().fillna(method='bfill').fillna(method='ffill')))
<ipython-input-148-c1fb62223757>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data90['SpO2(%)'] = data90['SpO2(%)'].apply(lambda x: np.array(pd.Series(x,dtype=float).interpolate().fillna(method='bfill').fillna(method='ffill')))
<ipython-input-148-c1fb62223757>:3: SettingWithC

In [151]:
data90['Percentage HR nan'] = data90['HR(bpm)'].apply(lambda x: np.isnan(x.astype(float)).sum()/len(x))
data90['Percentage SpO2 nan'] = data90['SpO2(%)'].apply(lambda x: np.isnan(x.astype(float)).sum()/len(x))
data90['Percentage NIBP nan'] = data90['NIBP(mmHg)'].apply(lambda x: np.isnan(x.astype(float)).sum()/len(x))

<ipython-input-151-a52c673fd8a8>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data90['Percentage HR nan'] = data90['HR(bpm)'].apply(lambda x: np.isnan(x.astype(float)).sum()/len(x))
<ipython-input-151-a52c673fd8a8>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data90['Percentage SpO2 nan'] = data90['SpO2(%)'].apply(lambda x: np.isnan(x.astype(float)).sum()/len(x))
<ipython-input-151-a52c673fd8a8>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using 

In [152]:
data90

,MeetTijd,HR(bpm),SpO2(%),NIBP(mmHg),IcNaOk,Length_MeetTijd,Percentage HR nan,Percentage SpO2 nan,Percentage NIBP nan
1,"[15:32:32, 15:33:32, 15:34:33, 15:35:33, 15:36...","[51.0, 49.0, 57.0, 49.0, 55.0, 50.0, 56.0, 49....","[92.0, 92.0, 87.5, 93.0, 93.0, 93.0, 92.0, 95....","[118.0, 118.0, 118.0, 120.0, 120.0, 120.0, 120...",0.0,146,0.0,0.0,0.0
2,"[08:45:22, 08:46:23, 08:47:23, 08:48:24, 08:49...","[79.0, 75.0, 71.0, 88.0, 88.0, 84.0, 66.0, 66....","[98.0, 98.0, 98.0, 97.33333333333333, 96.66666...","[129.0, 129.0, 129.0, 129.0, 133.0, 137.0, 141...",0.0,200,0.0,0.0,0.0
3,"[08:40:08, 08:41:08, 08:42:09, 08:43:09, 08:44...","[102.0, 99.0, 99.0, 101.0, 96.0, 101.0, 99.0, ...","[93.0, 95.58, 96.16, 96.08, 97.83, 96.66, 97.7...","[150.0, 158.0, 2.0, 151.0, 2.0, 2.0, 152.0, 15...",0.0,178,0.0,0.0,0.0
4,"[09:09:53, 09:10:53, 09:11:54, 09:12:54, 09:13...","[56.0, 57.0, 56.0, 58.0, 58.0, 57.0, 57.0, 63....","[97.0, 97.0, 98.0, 97.0, 98.0, 98.0, 97.5, 98....","[136.0, 131.0, 131.0, 131.0, 131.0, 131.0, 144...",0.0,75,0.0,0.0,0.0
5,"[13:45:16, 13:46:17, 13:47:17, 13:48:18, 13:49...","[79.0, 84.0, 78.0, 78.0, 79.0, 79.0, 81.0, 79....","[97.0, 97.0, 97.0, 97.0, 97.0, 97.0, 97.0, 97....","[115.0, 113.0, 113.0, 113.0, 108.0, 108.0, 108...",0.0,164,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...
8327,"[13:56:40, 13:57:10, 13:57:40, 13:58:11, 13:58...","[71.0, 71.0, 71.0, 71.0, 71.0, 71.0, 78.0, 75....","[100.0, 100.0, 99.0, 100.0, 100.0, 100.0, 100....","[138.0, 138.0, 138.0, 138.0, 138.0, 138.0, 138...",1.0,126,0.0,0.0,0.0
8329,"[17:26:10, 17:26:40, 17:27:11, 17:27:41, 17:28...","[80.0, 80.0, 80.0, 78.0, 82.0, 79.0, 77.0, 78....","[99.0, 100.0, 99.0, 99.0, 99.0, 99.0, 99.0, 99...","[90.0, 90.0, 94.0, 94.0, 94.0, 94.0, 94.0, 94....",1.0,190,0.0,0.0,0.0
8331,"[22:36:56, 22:37:27, 22:37:57, 22:38:27, 22:38...","[72.0, 71.0, 70.0, 70.0, 68.0, 68.0, 67.0, 66....","[100.0, 100.0, 100.0, 100.0, 100.0, 100.0, 100...","[160.0, 124.0, 124.0, 124.0, 124.0, 124.0, 124...",1.0,177,0.0,0.0,0.0
8333,"[19:53:44, 19:54:15, 19:54:45, 19:55:16, 19:55...","[125.0, 131.0, 128.0, 125.0, 126.0, 123.0, 128...","[97.0, 97.0, 98.0, 98.0, 98.0, 97.0, 97.0, 98....","[119.0, 119.0, 119.0, 119.0, 119.0, 119.0, 119...",1.0,176,0.0,0.0,0.0
